**Group members:** Ali Omarov, Simon Ruber, Esra Gücükbel

## Exercise 2

### a) Write a program in a programming language of your choice that connects to a Postgresql database and retrieves and displays data from it.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import datetime

In [2]:
# install and import library for postgresql connection
# pip install psycopg2
import psycopg2

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `apple_db` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS apple_db;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE apple_db;'

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 10.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package cron.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../0-cron_3.0pl1-128.1ubuntu1_amd64.deb ...
Unpacking cron (3.0pl1-128.1ubuntu1) ...
Selecting previously unselected package logrotate.
Preparing to unpack .../1-logrotate_3.11.0-0.1ubuntu1_amd64.deb ...
Unpacking logrotate (3.11.0-0.1ubuntu1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../2-netbase_5.4_all.deb ...
Unpacking netbase (5.4) ...
Selecting previously unselected pac

### Setup necessary environment variables

In [5]:
%env TFIO_DEMO_DATABASE_NAME=apple_db
%env TFIO_DEMO_DATABASE_HOST=localhost
%env TFIO_DEMO_DATABASE_PORT=5432
%env TFIO_DEMO_DATABASE_USER=postgres
%env TFIO_DEMO_DATABASE_PASS=postgres

env: TFIO_DEMO_DATABASE_NAME=apple_db
env: TFIO_DEMO_DATABASE_HOST=localhost
env: TFIO_DEMO_DATABASE_PORT=5432
env: TFIO_DEMO_DATABASE_USER=postgres
env: TFIO_DEMO_DATABASE_PASS=postgres


### Creating DB

In [6]:
# Starting connection
connection_config = "host=localhost port=5432 dbname=apple_db user=postgres password= postgres"
conn = psycopg2.connect("host=localhost port=5432 dbname=apple_db user=postgres password= postgres")
cur = conn.cursor()

In [7]:
cur.execute("""DROP TABLE IF EXISTS STOCK """)
cur.execute("""CREATE TABLE STOCK (
  Date DATE,
  Low_Value REAL,
  High_Value REAL,
  Day_End_Value REAL,
  Trade_Volume INT)""")
conn.commit()

In [8]:
# In case problem occurs
# cur.execute('ROLLBACK')
# conn.commit()

In [9]:
# We need to "DBS Assignment 09" folder as shortcut to our drives to work seamlessly.
with open("/content/drive/MyDrive/DBS Assignment 09/apple.csv", 'r') as f:
    reader = csv.reader(f)
    next(reader) # Skip the header row.
    for row in reader:
        row[0] = datetime.date(int('20'+row[0][6:8]),int(row[0][3:5]),int(row[0][0:2]))
        cur.execute(
        "INSERT INTO STOCK VALUES (%s, %s, %s, %s, %s)", row)
conn.commit()

### b) Modify your program so that the data is changed by the program before it is displayed.

In [10]:
def get_records(conn_config, sql_statement):
    """ query data from the Stock table """
    conn = None
    try:
        conn =  psycopg2.connect(conn_config)
        cur = conn.cursor()
        cur.execute(sql_statement)
        print("The number of records: ", cur.rowcount)
        row = cur.fetchone()

        while row is not None:
            print(row)
            row = cur.fetchone()

        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [11]:
sql_get_all_records = "SELECT * FROM STOCK"
get_records(connection_config, sql_get_all_records)

The number of records:  22
(datetime.date(2017, 1, 2), 109.3, 110.8, 110.3, 2330417)
(datetime.date(2017, 1, 3), 110.11, 112.09, 110.68, 882614)
(datetime.date(2017, 1, 4), 110.53, 111.7, 111.03, 449615)
(datetime.date(2017, 1, 5), 109.4, 110.73, 110.0, 746100)
(datetime.date(2017, 1, 6), 109.8, 112.0, 111.61, 700163)
(datetime.date(2017, 1, 9), 111.6, 113.1, 112.46, 1297576)
(datetime.date(2017, 1, 10), 111.8, 112.9, 112.75, 1691504)
(datetime.date(2017, 1, 11), 111.83, 113.86, 112.17, 1521071)
(datetime.date(2017, 1, 12), 110.94, 112.6, 111.76, 1073743)
(datetime.date(2017, 1, 13), 111.6, 112.5, 111.76, 1149538)
(datetime.date(2017, 1, 16), 111.72, 112.45, 112.0, 861942)
(datetime.date(2017, 1, 17), 110.0, 112.29, 112.08, 848050)
(datetime.date(2017, 1, 18), 112.0, 112.7, 112.35, 528968)
(datetime.date(2017, 1, 19), 111.99, 113.09, 112.71, 778900)
(datetime.date(2017, 1, 20), 112.14, 113.1, 112.14, 610841)
(datetime.date(2017, 1, 23), 111.04, 112.5, 111.74, 366744)
(datetime.date(201

In [12]:
sql_get_low_higher_111_records = "SELECT * FROM STOCK WHERE Low_Value > 111"
get_records(connection_config, sql_get_low_higher_111_records)

The number of records:  14
(datetime.date(2017, 1, 9), 111.6, 113.1, 112.46, 1297576)
(datetime.date(2017, 1, 10), 111.8, 112.9, 112.75, 1691504)
(datetime.date(2017, 1, 11), 111.83, 113.86, 112.17, 1521071)
(datetime.date(2017, 1, 13), 111.6, 112.5, 111.76, 1149538)
(datetime.date(2017, 1, 16), 111.72, 112.45, 112.0, 861942)
(datetime.date(2017, 1, 18), 112.0, 112.7, 112.35, 528968)
(datetime.date(2017, 1, 19), 111.99, 113.09, 112.71, 778900)
(datetime.date(2017, 1, 20), 112.14, 113.1, 112.14, 610841)
(datetime.date(2017, 1, 23), 111.04, 112.5, 111.74, 366744)
(datetime.date(2017, 1, 25), 111.7, 113.65, 113.65, 1221101)
(datetime.date(2017, 1, 26), 113.2, 114.38, 114.0, 1302214)
(datetime.date(2017, 1, 27), 113.7, 114.46, 114.03, 739897)
(datetime.date(2017, 1, 30), 112.9, 113.99, 113.35, 1268146)
(datetime.date(2017, 1, 31), 111.78, 113.5, 111.91, 988174)


In [16]:
# Query with date
sql_get_late_records = "SELECT * FROM STOCK WHERE Date > '2017-01-17'; "
get_records(connection_config, sql_get_late_records)

The number of records:  10
(datetime.date(2017, 1, 18), 112.0, 112.7, 112.35, 528968)
(datetime.date(2017, 1, 19), 111.99, 113.09, 112.71, 778900)
(datetime.date(2017, 1, 20), 112.14, 113.1, 112.14, 610841)
(datetime.date(2017, 1, 23), 111.04, 112.5, 111.74, 366744)
(datetime.date(2017, 1, 24), 110.9, 111.73, 111.51, 734456)
(datetime.date(2017, 1, 25), 111.7, 113.65, 113.65, 1221101)
(datetime.date(2017, 1, 26), 113.2, 114.38, 114.0, 1302214)
(datetime.date(2017, 1, 27), 113.7, 114.46, 114.03, 739897)
(datetime.date(2017, 1, 30), 112.9, 113.99, 113.35, 1268146)
(datetime.date(2017, 1, 31), 111.78, 113.5, 111.91, 988174)
